<h1>Digit Recognization With Tensorflow</h1>

<h2 style="font-size: 20px;">Import Necessary Libraries and packages</h2>

In [31]:
!pip install keras-tuner -q 

In [45]:
!pip install --upgrade tensorflow -q

In [33]:
import os
import datetime
import numpy as np
import pandas as pd
import keras_tuner as kt
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import callbacks
from tensorflow.keras import layers
from typing import Union, Tuple, Optional
print("tensorflow version: ", tf.__version__)

tensorflow version:  2.6.4


In [34]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


<h2 style="font-size: 20px;">Setting The Notebook</h2>

In [44]:
# Setting the graph style
plt.rc('figure', autolayout=True)
plt.rc(
    'axes', titleweight='bold', 
    titlesize=20, labelweight=700,
    labelsize=13
    )
plt.rc('font', size=15)

def set_seeds(seed: int=0):
    """Sets the Seed into order to get the same result on every code run"""
    os.environ["PYTHONHASHSEED"] = str(seed)
    tf.random.set_seed(seed)
    np.random.seed(seed)
    os.environ["TF_DETERMINISTIC"] = str(seed)
    os.environ["CUDA_VISIBLE_DEVICES"]="-1"

# Call the set_seeds with default parameter
set_seeds()

try:
    tfu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
    strategy = tf.distribute.TPUStrategy(tfu)
except ValueError:
    gpus = tf.config.list_logical_devices('GPU')
    print(f"Number of gpus: {len(gpus)}")
    if len(gpus) > 0:
        strategy = tf.distribute.MirroredStrategy(gpus)
    else:
        strategy = tf.distribute.get_strategy()
print(f"Number of Accelerator: {strategy.num_replicas_in_sync}")

Number of gpus: 0
Number of Accelerator: 1


In [36]:
train_nrow = 42000
test_nrow = 28000
training_size = train_nrow - test_nrow
BATCH_SIZE_PER_REPLICA = 32
batch_size = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync
steps_per_epoch = training_size // batch_size
validation_steps = test_nrow // batch_size

<h2 style="font-size: 20px;">Above Dataset</h2>
<p style="font-size: 15px;">
The data files train.csv and test.csv contain gray-scale images of hand-drawn digits, from zero through nine.

Each image is 28 pixels in height and 28 pixels in width, for a total of 784 pixels in total. Each pixel has a single pixel-value associated with it, indicating the lightness or darkness of that pixel, with higher numbers meaning darker. This pixel-value is an integer between 0 and 255, inclusive.

The training data set, (train.csv), has 785 columns. The first column, called "label", is the digit that was drawn by the user. The rest of the columns contain the pixel-values of the associated image.

Each pixel column in the training set has a name like pixelx, where x is an integer between 0 and 783, inclusive. To locate this pixel on the image, suppose that we have decomposed x as x = i * 28 + j, where i and j are integers between 0 and 27, inclusive. Then pixelx is located on row i and column j of a 28 x 28 matrix, (indexing by zero).
</p>

<h2 style="font-size: 20px;">Load Csv File Into Pandas DataFrame</h2>

In [37]:
try:
    from google.colab import drive
    # Mount google files
    drive.mount('/gdrive')

    # list the files from drive
    folder_path = '../gdrive/MyDrive/Datasets/MNIST_data'
    
except ImportError:
    folder_path = '../input/digit-recognizer'

os.listdir(folder_path)

['sample_submission.csv', 'train.csv', 'test.csv']

In [38]:
# Data Path
train_data_path = folder_path + '/train.csv'
test_data_path = folder_path + '/test.csv'

# Loading dataset into pandas 
train_df = pd.read_csv(train_data_path)
test_df = pd.read_csv(test_data_path)

<h2 style="font-size: 20px;">View The Dataframe</h2>

In [39]:
# First Rows in the train data
print(train_df.shape)
train_df.head()

(42000, 785)


,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [40]:
# First Rows in the test data
print(test_df.shape)
test_df.head()

(28000, 784)


,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


<h2 style="font-size: 20px;">Train Data Split Into Train and Validation data</h2>

In [41]:
mask = np.random.randn(len(train_df)) > 0.8
train_data = train_df[mask]
valid_data = train_df[~mask]
print(f"Train_data shape: {train_data.shape}\nValid_data: {valid_data.shape}")

Train_data shape: (8828, 785)
Valid_data: (33172, 785)


<h2 style="font-size: 20px;">Create Dataset Generator</h2>

In [42]:
class DatasetGenerator:
    def __init__(self, batch_size: int = batch_size, shuffle: int = False, epoch: int = 10) -> None:
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.epoch = epoch

    def __call__(self, df: pd.DataFrame) -> any:
        if self.shuffle:
            df_ = tf.constant([
                row[1:].values / 255
                for _, row in df.iterrows()
                ])
            df_ = tf.reshape(df_, [len(df), 28, 28, 1])
            labels = df.iloc[:, 0]
            data = tf.data.Dataset.from_tensor_slices((df_, labels)).batch(self.batch_size).shuffle(1000)
        else:
            df_ = tf.constant([
                row.values.reshape(28, 28) /255
                for _, row in df.iterrows()
                ])
            df_ = tf.reshape(df_, [len(df), 28, 28, 1])
            data = tf.data.Dataset.from_tensor_slices((df_)).batch(self.batch_size)
        return data
    
    def AsGraphDefInternal():
        pass

<h2 style="font-size: 20px;">Convert The Data To tensor</h2>

In [43]:
# Change the dataframes to generator 
train_data = DatasetGenerator(shuffle=True, batch_size=batch_size)(train_data)
validation_data = DatasetGenerator(shuffle=True, batch_size=batch_size)(valid_data)
test_dataset = DatasetGenerator()(test_df)
train_data

KeyboardInterrupt: 

<h2 style="font-size: 20px;">Exploring Datasets</h2>

In [ ]:
# Exploring One of the image in the train_dataset generator
img, label = next(iter(train_data))

In [ ]:
print("image Datatype: ", type(img[0]))
print("label data type: ", type(label[1]))
print("label: ", set(label.numpy()))

In [ ]:
# For exploratory purpose change the image to numpy array
np_img = np.array(img)
IMG_SHAPE = np_img.shape
print("Image shape: ", IMG_SHAPE)

<h2 style="font-size: 20px;">Digits Visualization</h2>

In [ ]:
# Visualizing of the one image in the test_dataset
img = next(iter(test_dataset))
cmap = plt.get_cmap('magma')
plt.imshow(tf.squeeze(img[0]), cmap=cmap)
plt.axis('off')
plt.show()


In [ ]:
# Create Subplot instance
fig, ax = plt.subplots(figsize=(8, 8))

# Number of columns and rows
NROW = 3
NCOL = 3

# loop over the range of NROW * NCOL starting from 1
for i in range(1, NROW * NROW + 1):
    fig.add_subplot(NROW, NCOL, i)

    # Get the image from the generated index
    img, label = next(iter(train_data))

    # Plot it
    plt.imshow(tf.squeeze(img[0]), 
        cmap=cmap
        )
    plt.axis('off')
    plt.title(label[0].numpy())
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax.set_axis_off()

<h2 style="font-size: 20px;">Modeling</h2>

In [ ]:
EPOCHS = 100
INPUT_SHAPE = (28, 28, 1)

In [ ]:
def conv_block(
        filters, 
        kernel_size=(2, 2), 
        activation='relu',
        dropout=False,
        dropout_rate=0.5,
        pooling=None,
        pool_size=(2, 2),
        input_shape=None):
        
    if input_shape:
        conv = layers.Conv2D(
            filters=filters,
            kernel_size=kernel_size,
            input_shape=input_shape,
            activation=activation)
    else:
        conv = layers.Conv2D(
             filters=filters,
             kernel_size=kernel_size,
             activation=activation)
            
    if pooling == 'average':
        pool = layers.AveragePooling2D(
                pool_size=pool_size
               )
    elif pooling == 'max':
        pool = layers.MaxPooling2D(
                pool_size=pool_size
               )
    else:
        pass 
            
    if dropout:
        drop_out = layers.Dropout(rate=dropout_rate)
        
        
    def _forward(X):
        output = conv(X)
            
        if pooling:
            output = pool(output)
            
        if dropout:
            output = drop_out(output)
                
        return output 
        
    return _forward

In [ ]:
class ConvModel(keras.Model):
    # Construct 
    def __init__(self) -> None:
        
        super().__init__()
        # ==== BASE =====
        # Input layer.
        self.input_conv = conv_block(64, input_shape=INPUT_SHAPE)
        
        # Hidden layers.
        self.hidden_conv_1 = conv_block(128, pooling='max', dropout=True)
        self.hidden_conv_2 = conv_block(128, pooling='max', dropout=True)

        
        # ==== HEAD ====
        self.flatten = layers.Flatten()
        self.linear_1 = layers.Dense(10, activation='softmax')
        
        
    def call(self, X) -> tf.Tensor:
        # ==== BASE =====
        # Input layer.
        input_layer = self.input_conv(X)
        
        # Hidden layers.
        hidden_conv_1 = self.hidden_conv_1(input_layer)
        hidden_conv_2 = self.hidden_conv_2(hidden_conv_1)
        
        # ==== HEAD ====
        flatten = self.flatten(hidden_conv_2)
        linear_1 = self.linear_1(flatten)
        return linear_1

In [ ]:
with strategy.scope():
    model = ConvModel()
    model.compile(
        loss=keras.losses.SparseCategoricalCrossentropy(),
        optimizer='adam',
        metrics=['accuracy']
        )
    model.build(input_shape=IMG_SHAPE)
    model.fit(train_data, epochs=100, validation_data=validation_data)

<h2 style="font-size: 20px;">Submission</h2>

In [ ]:
# Make predictions
prob  = conv_model.predict(test_dataset)
# Get the highest value from probability
pred = tf.argmax(prob, 1)

In [ ]:
submission=pd.read_csv(folder_path + '/sample_submission.csv')
submission['Label']=pred
submission.to_csv('prediction20_conv.csv', index=False)